In [4]:
import json
from llama_models.llama3.reference_impl.generation import Llama
from llama_models.llama3.api.datatypes import UserMessage, SystemMessage
import os
import sys
from llama_models.llama3.api.datatypes import (
    UserMessage,
    SystemMessage,
    CompletionMessage,
    StopReason,
)
from llama_models.llama3.reference_impl.generation import Llama
import pandas as pd
from io import StringIO
import numpy as np
import re
import requests
import ollama

# Funciones

In [101]:
def transformar_texto(texto, marca):
    if texto is None or (isinstance(texto, float) and np.isnan(texto)):
        return texto
    
    if marca.lower() == "adidas":
        # Transformación original para adidas
        if isinstance(texto, (list, np.ndarray)):
            texto = ", ".join(map(str, texto))
        else:
            texto = str(texto)
        texto = texto.strip("[]")
        texto = re.sub(r",\s*", '} {', texto)
        texto = '{' + texto + '}'
        return texto
    
    elif marca.lower() == "nike":
        # Transformación específica para nike
        if isinstance(texto, list):
            # Elimina claves con valores irrelevantes
            texto_limpio = [
                {k.strip(): v.strip() for k, v in d.items() if k.strip() not in ['\xa0']}
                for d in texto
                if isinstance(d, dict)
            ]
            # Filtra elementos vacíos o irrelevantes
            texto_limpio = [d for d in texto_limpio if d]
            return texto_limpio
        return texto  # Si no es lista, regresa el texto original

    else:
        raise ValueError(f"Marca '{marca}' no reconocida. Usa 'adidas' o 'nike'.")


In [15]:
def generar_prompt_llama(descripcion, etiquetas):
    prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
    prompt += "Eres un asistente que extrae información de descripciones de productos y las organiza en una tabla.\n"
    prompt += "<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
    prompt += f"Extrae la siguiente información de la descripción del producto y etiquétala según las etiquetas: {', '.join(etiquetas)}.\n\n"
    prompt += f"Descripción: {descripcion}\n\n"
    prompt += "Proporciona la información en una tabla con las columnas correspondientes a cada etiqueta.\n"
    prompt += "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    prompt += "| " + " | ".join(etiquetas) + " |\n"
    prompt += "| " + " | ".join(['---'] * len(etiquetas)) + " |\n"
    prompt += "|"
    return prompt


In [20]:
def obtener_respuesta_llama(dialogo):
    try:
        # Ruta al modelo descargado (ajusta esta ruta según corresponda)
        ruta_modelo = os.path.expanduser('~/.llama/checkpoints/Meta-Llama3.1-8B-Instruct')

        # Cargar el modelo Llama 3.1 8B
        modelo = Llama.build(
            ckpt_dir=ruta_modelo,
            max_seq_len=1024,        # Ajusta según sea necesario
            max_batch_size=1,        # Número de prompts a procesar simultáneamente
            model_parallel_size=1,
        )

        # Generar la respuesta utilizando el diálogo
        resultado = modelo.chat_completion(
            dialog=dialogo,
            max_gen_len=512,         # Máxima longitud de generación
            temperature=0.6,         # Ajusta según sea necesario
            top_p=0.9,               # Ajusta según sea necesario
        )

        # Obtener el mensaje generado por el asistente
        mensaje_asistente = resultado.generation
        return mensaje_asistente.content

    except Exception as e:
        print(f"Error al generar la respuesta: {e}")
        return None

In [22]:
def obtener_respuesta_ollama(prompt):
    response = ollama.chat(
        model="llama3.2:latest",
        messages = [
           {
               "role":"user",
               "content": prompt
           } 
        ]
    )
    # La respuesta es un generador; concatenamos las partes
    return response

In [23]:
def procesar_respuesta(respuesta_texto, etiquetas):
    try:
        # Buscar el inicio de la tabla
        inicio_tabla = respuesta_texto.find('|')
        if inicio_tabla != -1:
            # Extraer la tabla desde el primer '|'
            tabla_texto = respuesta_texto[inicio_tabla:].strip()
            # Extraer solo las líneas que contienen '|'
            lineas = tabla_texto.split('\n')
            lineas_tabla = []
            for linea in lineas:
                if '|' in linea:
                    # Eliminar los '|' iniciales y finales y espacios extra
                    linea = linea.strip().strip('|').strip()
                    lineas_tabla.append(linea)
                else:
                    # Detenerse si la línea no contiene '|'
                    break
            tabla_completa = '\n'.join(lineas_tabla)
            # Convertir la tabla Markdown a un DataFrame
            df = pd.read_csv(StringIO(tabla_completa), sep='|', engine='python', skipinitialspace=True)
            # Limpiar nombres de columnas y datos
            df.columns = [col.strip() for col in df.columns]
            for col in df.columns:
                if df[col].dtype == object:
                    df[col] = df[col].str.strip()
            # Resetear el índice
            df = df.reset_index(drop=True)
            return df
        else:
            print("No se encontró una tabla en la respuesta.")
            return None
    except Exception as e:
        print(f"Error al procesar la tabla: {e}")
        return None


# Lectura de datos desde la api

In [63]:
url ="https://scraping-firestore-178159629911.us-central1.run.app//v1/scraping/"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(response.status_code)
else:
    print(f'Error: {response.status_code}')

df_raw = pd.DataFrame(data)

200


In [125]:
df_raw.head()

,id,details,store,manufacturer,url,title,regularPrice,undiscounted_price,description,category,createdAt,characteristics,gender
0,046zSiHm8Cz0fZYwMJlL,[],adidas,adidas,https://www.adidas.co/tenis-duramo-sl/IF7884.html,Tenis Duramo SL,$379.950,$265.965,Siente la ligereza y velocidad. Si estás listo...,Mujer • Running,"{'_seconds': 1731975445, '_nanoseconds': 42700...",NaN,NaN
1,08sjncACSjSvg2t9DS73,"[Horma clásica, Parte superior sintética, Forr...",adidas,adidas,https://www.adidas.co/tenis-adizero-adios-pro-...,Tenis ADIZERO ADIOS PRO 3,$1.299.950,$909.965,Los Adizero Adios Pro 3 son la máxima expresió...,Mujer • Running,"{'_seconds': 1731975445, '_nanoseconds': 42700...",NaN,NaN
2,0AqheRhKT2lhm7puBVCF,"[Ajuste clásico, Sistema de amarre de cordones...",adidas,adidas,https://www.adidas.co/tenis-ultraboost-22/GX55...,TENIS ULTRABOOST 22,$799.950,NA,Hemos analizado 1.200.000 pisadas para que Ult...,Mujer • Running,"{'_seconds': 1731975445, '_nanoseconds': 42700...",NaN,NaN
3,0Di5KVVcvU0QsWRxB1iE,[{' ': '<b>&nbsp;</b>'}],nike,nike,https://www.nike.com.co/nike-revolution-7-fb22...,Nike Revolution 7,$ 389.950,NA,Cargamos el Revolution 7 con el tipo de amorti...,Calzado de correr en pavimento para mujer,"{'_seconds': 1731965768, '_nanoseconds': 30000...",[],Mujer
4,0Gvnv9unc1EV4XpFbCQN,[{'Características principales': '<b>Caracterí...,nike,nike,https://www.nike.com.co/nike-winflo-11-calzado...,Nike Winflo 11,$ 584.950,NA,El Winflo 11 es el calzado con una pisada bala...,Calzado de correr en pavimento para mujer,"{'_seconds': 1731965768, '_nanoseconds': 30000...",[Parte superior de malla diseñada estratégicam...,Mujer


## Paso 1: Definir las etiquetas dinámicamente
Creamos una lista de etiquetas que pueden cambiar según las necesidades

In [64]:
etiquetas = ["cordones", "textil exterior", "textil interior", "suela", "peso y/o talla", "eco diseñado si o no", "color", "identificador", "nombre de deporte",'genero Mujer/Hombre/MIXTO']


## Paso 2: procesar info de Adidas

In [65]:
df_adidas = df_raw[df_raw['store'] == 'adidas']

In [102]:
# Lista de descripciones de productos
df_adidas['details_transformado'] = df_adidas['details'].apply(lambda x: transformar_texto(x, 'adidas'))


C:\Users\Usuario\AppData\Local\Temp\ipykernel_5352\4001796790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adidas['details_transformado'] = df_adidas['details'].apply(lambda x: transformar_texto(x, 'adidas'))


In [ ]:
# Lista de descripciones de productos
'''descripciones = [
    "[{'text': 'Ajuste clásico'}, {'text': 'Sistema de amarre de cordones'}, {'text': 'Parte superior textil'}, {'text': 'Mediasuela Cloudfoam'}, {'text': 'Forro interno textil'}, {'text': 'Peso: 239 gramos (talla CO 37,5)'}, {'text': 'Caída mediasuela: 9 mm (talón: 28 mm / antepié: 19 mm)'}, {'text': 'Suela de caucho'}, {'text': 'La parte superior contiene al menos un 50% de material reciclado'}, {'text': 'Color del artículo: Dark Blue / Core Black / Gold Metallic'}, {'text': 'Número de artículo: IE0747'}]",
    "[{'text': 'Sistema de amarre de cordones'}, {'text': 'Exterior textil con refuerzos sintéticos sin costuras'}, {'text': 'Lengüeta reforzada'}, {'text': 'Amortiguación Lightstrike'}, {'text': 'Capa protectora de TPU'}, {'text': 'Suela de caucho Continental™ Rubber'}, {'text': 'El exterior contiene al menos un 50 % de material reciclado'}, {'text': 'Color del artículo: Core Black / Grey Three / Grey Two'}, {'text': 'Número de artículo: HR1182'}]",
    "[{'text': 'Corte clásico'}, {'text': 'Sistema de amarre de cordones'}, {'text': 'Exterior de malla sándwich'}, {'text': 'Revestimientos sin costuras que brindan soporte'}, {'text': 'Talón suave'}, {'text': 'Amortiguación LIGHTMOTION'}, {'text': 'Suela de caucho'}, {'text': 'El exterior contiene al menos un 50 % de material reciclado'}, {'text': 'Color del artículo: Light Grey / Screaming Orange / Solar Gold'}, {'text': 'Número de artículo: HP2375'}]",
    "[{'text': 'Ajuste clásico'}, {'text': 'Sistema de amarre de cordones'}, {'text': 'Parte superior textil'}, {'text': 'Forro interno textil'}, {'text': 'Suela multiterreno de caucho'}, {'text': 'Peso: 327 gramos (talla CO 40)'}, {'text': 'Caída mediasuela: 9 mm (talón: 29 mm / antepié: 19 mm)'}, {'text': 'Contienen al menos un 20% de material reciclado'}, {'text': 'Color del artículo: Halo Silver / Green Spark / Grey Five'}, {'text': 'Número de artículo: IG1416'}]",
    "[{'text': 'Ajuste clásico'}, {'text': 'Sistema de amarre de cordones'}, {'text': 'Exterior de malla'}, {'text': 'Diseño cómodo para impulsarte hacia adelante'}, {'text': 'Forro interno textil'}, {'text': 'Mediasuela de EVA'}, {'text': 'Peso: 236 g (Talla COL 36,5)'}, {'text': 'Caída mediasuela: 10 mm (talón: 32 mm / antepié: 22 mm)'}, {'text': 'Suela Adiwear'}, {'text': 'El exterior contiene al menos un 50 % de material reciclado'}, {'text': 'Color del artículo: Core Black / Cloud White / Grey Six'}, {'text': 'Número de artículo: ID5258'}]",
    "[{'text': 'Ajuste clásico'}, {'text': 'Sistema de amarre de cordones'}, {'text': 'Exterior de malla'}, {'text': 'Diseño cómodo para impulsarte hacia adelante'}, {'text': 'Forro interno textil'}, {'text': 'Mediasuela de EVA'}, {'text': 'Peso: 236 g (Talla COL 36,5)'}, {'text': 'Caída mediasuela: 10 mm (talón: 32 mm / antepié: 22 mm)'}, {'text': 'Suela Adiwear'}, {'text': 'El exterior contiene al menos un 50 % de material reciclado'}, {'text': 'Color del artículo: Cloud White / Silver Metallic / Crystal White'}, {'text': 'Número de artículo: ID5257'}]",
    "[{'text': 'Ajuste clásico'}, {'text': 'Sistema de amarre de cordones'}, {'text': 'Parte superior textil'}, {'text': 'Mediasuela Cloudfoam'}, {'text': 'Forro interno textil'}, {'text': 'Peso: 286 gramos (talla CO 40)'}, {'text': 'Caída mediasuela: 10 mm (talón: 29,2 mm / antepié: 19,2 mm)'}, {'text': 'Suela de caucho'}, {'text': 'La parte superior contiene al menos un 50% de material reciclado'}, {'text': 'Color del artículo: Shadow Red / Green Spark / Better Scarlet'}, {'text': 'Número de artículo: IE0740'}]"
]'''

In [103]:
df_adidas[['details_transformado','description','category']].head()

,details_transformado,description,category
0,{},Siente la ligereza y velocidad. Si estás listo...,Mujer • Running
1,{Horma clásica} {Parte superior sintética} {Fo...,Los Adizero Adios Pro 3 son la máxima expresió...,Mujer • Running
2,{Ajuste clásico} {Sistema de amarre de cordone...,Hemos analizado 1.200.000 pisadas para que Ult...,Mujer • Running
5,{Ajuste clásico} {Cierre de cordones} {Parte s...,Haz tus mejores 10k con nuestros nuevos tenis ...,Mujer • Running
7,{Ajuste clásico} {Sistema de amarre de cordone...,Haz tus mejores 10k con nuestros nuevos tenis ...,Hombre • Running


In [38]:
# Crear un diccionario con las llaves deseadas
productos = [
    {
        "details": row['details_transformado'],
        "description": row['description'],
        "category": row['category']
    }
    for _, row in df_adidas[:10].iterrows() 
    if row['details_transformado'] != '{}'
]


## Paso 3: Integrar con la API de Llama 3.1


### Prompt

In [60]:
productos


[{'details': '{Horma clásica} {Parte superior sintética} {Forro interno textil} {Varillas ENERGYRODS 2.0 que limitan la pérdida de energía} {Amortiguación Lightstrike Pro} {Peso: 183 g (talla CO 37} {5)} {Caída mediasuela: 6 mm (talón: 38 mm / antepié: 32 mm)} {La parte superior contiene al menos un 50 % de material reciclado} {Color del artículo: Pink Spark / Aurora Met. / Sandy Pink} {Número de artículo: ID3612}',
  'description': "Los Adizero Adios Pro 3 son la máxima expresión de los productos Adizero Racing. Fueron diseñados con y para atletas para lograr hazañas increíbles. Estos tenis de running adidas están diseñados para optimizar la eficiencia del running. Nuestras varillas ENERGYRODS 2.0 de carbono ofrecen ligereza y firmeza para pasos ágiles y eficientes. La tecnología LIGHTSTRIKE PRO ultraliviana amortigua cada paso con las tres capas de espuma resistente que te ayudan a mantener la energía a largo plazo. Todo sobre una delgada suela de caucho textil para un agarre extraor

In [56]:
def generar_prompt_ollama(producto, etiquetas):
    prompt = f"""
        Eres un asistente especializado en extraer información estructurada de descripciones de productos y organizarla en tablas. 
        Tu tarea es extraer la siguiente información de los detalles del producto y etiquetarla según las etiquetas: {', '.join(etiquetas)}.
        Si una etiqueta no tiene una correspondencia clara en los detalles, completa su valor con "null".

        Información del producto:
        - Details: {producto['details']}
        - Description: {producto['description']}
        - Category: {producto['category']}

        Proporciona la información en una tabla con las columnas correspondientes a cada etiqueta. 
        La tabla debe incluir **dos filas completas**:
        1. La primera fila contiene los nombres de las etiquetas.
        2. La segunda fila contiene los valores correspondientes etiquetados.

        Formato esperado de la respuesta:
        | {' | '.join(etiquetas)} |
        | {' | '.join(['---'] * len(etiquetas))} |
        | valor_1 | valor_2 | ... |
        
        Ejemplo:
        Si las etiquetas son "details", "description", y "category", y los valores extraídos son 
        "Zapatillas cómodas", "Hechas con materiales reciclados", y "Calzado", respectivamente, 
        tu respuesta debe ser:

        | details | description | category |
        | Zapatillas cómodas | Hechas con materiales reciclados | Calzado |

        Recuerda:
        - La respuesta debe contener dos filas completas.
        - Solo responde con la tabla y **no incluyas texto adicional**.

        Ahora, extrae y estructura la información del producto proporcionado:

        | {' | '.join(etiquetas)} |
        | {' | '.join(['---'] * len(etiquetas))} |
        |"""
    return prompt.strip()


### Generacion

In [57]:
dfs = []
for producto in productos:
    prompt = generar_prompt_ollama(producto, etiquetas)
    respuesta = obtener_respuesta_ollama(prompt)
    print(producto)
    print("Respuesta del modelo:")
    print(respuesta["message"]["content"])
    print("\n------------------------\n")
    df = procesar_respuesta(respuesta["message"]["content"], etiquetas)
    if df is not None:
        dfs.append(df)
    else:
        print("No se pudo extraer la tabla.\n")

{'details': '{Horma clásica} {Parte superior sintética} {Forro interno textil} {Varillas ENERGYRODS 2.0 que limitan la pérdida de energía} {Amortiguación Lightstrike Pro} {Peso: 183 g (talla CO 37} {5)} {Caída mediasuela: 6 mm (talón: 38 mm / antepié: 32 mm)} {La parte superior contiene al menos un 50 % de material reciclado} {Color del artículo: Pink Spark / Aurora Met. / Sandy Pink} {Número de artículo: ID3612}', 'description': "Los Adizero Adios Pro 3 son la máxima expresión de los productos Adizero Racing. Fueron diseñados con y para atletas para lograr hazañas increíbles. Estos tenis de running adidas están diseñados para optimizar la eficiencia del running. Nuestras varillas ENERGYRODS 2.0 de carbono ofrecen ligereza y firmeza para pasos ágiles y eficientes. La tecnología LIGHTSTRIKE PRO ultraliviana amortigua cada paso con las tres capas de espuma resistente que te ayudan a mantener la energía a largo plazo. Todo sobre una delgada suela de caucho textil para un agarre extraordin

In [112]:
if dfs:
    df_total = pd.concat(dfs, ignore_index=True)
else:
    print("No se pudo crear el DataFrame total.")

In [59]:
df_total

,cordones,textil exterior,textil interior,suela,peso y/o talla,eco diseñado si o no,color,identificador,nombre de deporte,genero Mujer/Hombre/MIXTO
0,Adizero Adios Pro 3,Parte superior sintética,Forro interno textil,Delgada suela de caucho textil,CO 37,Sí,Pink Spark / Aurora Met. / Sandy Pink,ID3612,Running,Mujer
1,Sistema de amarre de cordones,adidas Primeknit,null,Mediasuela BOOST,"Talla CO 37, 289 g",Si,Core Black / Cloud White,GX5591,Running,Mujer
2,Cierre de cordones,Parte superior de malla técnica,Amortiguación Lightstrike Pro,Suela de caucho Continental™,166 g (Talla COL 36 1/2),Contiene al menos un 20 % de material reciclado,Green Spark / Aurora Met. / Lucid Lemon,IG8206,Running,Mujer
3,Sistema de amarre de cordones,Parte superior de malla,Amortiguación Lightstrike Pro,Suela de caucho Continental™ Rubber,CO 40,Contienen al menos un 20% de material reciclado,Green Spark / Aurora Met. / Lucid Lemon,IG3134,Running,Mujer/Hombre/MIXTO
4,Sistema de amarre de cordones,Parte superior textil,Forro interno textil,"Mediasuela Cloudfoam, Suela de TPU",Talla CO 40,Sí,Halo Silver / Carbon / Core Black,ID8754,Running,Hombre
5,Si este artículo es personalizado,{Sistema de amarre de cordones para un ajuste ...,El exterior contiene al menos un 50% de materi...,Caucha,null,No,Ivory / Iron Metallic / Spark,IG3341,Running,Mujer
6,Sistema de amarre de cordones,Parte superior de malla,Forro interno textil,Mediasuela Cloudfoam,Talla CO 40,null,Cloud White / Core Black / Better Scarlet,IE8818,Running,Hombre
7,Sistema de amarre de cordones,Forro interno textil,mediasuela Dreamstrike+,Suela Adiwear,CO 37: 213 gramos,Contienen al menos un 20% de material reciclad...,Almost Yellow / Zero Metalic / Pink Spark,IE1072,Running,Mujer
8,Sistema de amarre de cordones,Exterior de malla con cuello acolchado,Forro interno textil,Mediasuela Bounce,null,Si,Shadow Violet / Legend Ink / Wonder Clay,IG0334,Running,Mujer


## Paso 5: Nike

In [67]:
df_nike = df_raw[df_raw['store'] == 'nike']

In [69]:
df_nike['details_transformado'] = df_nike['details'].apply(transformar_texto)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5352\989177854.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nike['details_transformado'] = df_nike['details'].apply(transformar_texto)


In [100]:
df_nike['details'].iloc[0]

[{'\xa0': '<b>&nbsp;</b>'}]

In [106]:
# Aplicar la transformación con el parámetro 'adidas'
df_nike['details_transformado'] = df_nike['details'].apply(lambda x: transformar_texto(x, 'nike'))

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5352\1496571984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nike['details_transformado'] = df_nike['details'].apply(lambda x: transformar_texto(x, 'nike'))


In [97]:
# Crear un diccionario con las llaves deseadas
productos_nike = [
    {
        "details": row['details_transformado'],
        "description": row['description'],
        "category": row['category']
    }
    for _, row in df_nike[:10].iterrows() 
    if row['details_transformado'] != '{}'
]

In [108]:
dfs_nike = []
for producto in productos_nike:
    prompt = generar_prompt_ollama(producto, etiquetas)
    respuesta = obtener_respuesta_ollama(prompt)
    print(producto)
    print("Respuesta del modelo:")
    print(respuesta["message"]["content"])
    print("\n------------------------\n")
    df_nike = procesar_respuesta(respuesta["message"]["content"], etiquetas)
    if df_nike is not None:
        dfs_nike.append(df_nike)
    else:
        print("No se pudo extraer la tabla.\n")

{'details': "{{'\\xa0': '<b>&nbsp;</b>'}}", 'description': 'Cargamos el Revolution 7 con el tipo de amortiguación suave y soporte que podría cambiar tu mundo del running. Elegante como siempre, cómodo cuando la goma se encuentra con la carretera y con alto rendimiento para el ritmo deseado, es una evolución de un favorito de los fanáticos que ofrece una conducción suave y tersa.', 'category': 'Calzado de correr en pavimento para mujer'}
Respuesta del modelo:
| cordones | textil exterior | textil interior | suela | peso y/o talla | eco diseñado si o no | color | identificador | nombre de deporte | genero Mujer/Hombre/MIXTO |
| Zapatillas cómodas | null | null | null | null | null | null | Revolution 7 | Correr | Mujer

------------------------

{'details': "{{'Características principales': '<b>Características principales</b>'}} {{'Datos del producto': '<b>Datos del producto</b>'}} {{'Soporte: neutro': '<b>Soporte: neutro</b>'}} {{'Ajuste adaptable': '<b>Ajuste adaptable</b>'}} {{'Datos 

In [111]:
if dfs_nike:
    df_total_nike = pd.concat(dfs_nike, ignore_index=True)
else:
    print("No se pudo crear el DataFrame total.")

In [110]:
df_total_nike

,cordones,textil exterior,textil interior,suela,peso y/o talla,eco diseñado si o no,color,identificador,nombre de deporte,genero Mujer/Hombre/MIXTO
0,Zapatillas cómodas,null,null,null,null,null,null,Revolution 7,Correr,Mujer
1,Zapatillas con fijación,Neutra,Textil sintético,Suela de goma,-,Si,Blanco/Rosa/Violeta/Verde,WINFLO 11,Correr en pavimento para mujer,Mujer/Hombre/MIXTO
2,Null,Null,Null,Null,Null,Si,Color fáciles de combinar,Invincible 3,Calzado de correr en pavimento para mujer,Mujer
3,Amortiguación: superalta,Nike ZoomX con forma de mecedora y espuma más ...,transpirable,Cuanta más amortiguación tengas en la planta d...,null,Si,Blanco/Sierra Verde,Invincible 3,Calzado de running en carretera para hombre,Hombre
4,Nike Grind,-,-,waffle,null,sí,-,-,Running,Hombre/MIXTO
5,Amortiguación: alta,Espuma,Textil,Suela con espuma,Pila alta,No,Blanco/ Negro,Structure 25,Running,Mujer/MIXTO
6,Características principales,La amortiguación máxima proporciona una comodi...,Hechas con materiales reciclados,Transpirabilidad contenida,Moderada,Negro,X,NaN,Calzado de correr en carretera para mujer,Mujer
7,Amortiguación: superalta,Nike ReactX Foam,Flyknit más adaptable,Suela curva,-,No,Negro/Blanco,InfinityRN 4,Calzado de running en carretera para hombre,Hombre/MIXTO
8,Responsividad: superalta,"ZoomX Foam, espuma más ligera y con mayor reto...",-,suela sin especificar,null,No,Color negro/negro escuro,Vomero 17,Calzado de running en carretera para hombre,Mujer/Hombre/MIXTO
9,La innovación es nuestra inspiración,transpirable y cómodo,superalta,alta,NaN,no,"negro, gris",Invincible 3,Calzado de correr en carretera para hombre,Hombre


## Nacion Runer

In [113]:
df_nr = df_raw[df_raw['store'] == 'nacionRunner']

In [116]:
df_nr.shape

(263, 13)

In [118]:
df_nr.columns

Index(['id', 'details', 'store', 'manufacturer', 'url', 'title',
       'regularPrice', 'undiscounted_price', 'description', 'category',
       'createdAt', 'characteristics', 'gender'],
      dtype='object')

In [119]:
# Aplicar la transformación con el parámetro 'adidas'
df_nr['details_transformado'] = df_nr['details'].apply(lambda x: transformar_texto(x, 'adidas'))

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5352\2945244906.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nr['details_transformado'] = df_nr['details'].apply(lambda x: transformar_texto(x, 'adidas'))


In [121]:
# Crear un diccionario con las llaves deseadas
productos_nr = [
    {
        "details": row['details_transformado'],
        "description": row['description'],
        "category": row['category']
    }
    for _, row in df_nr[:10].iterrows() 
    if row['details_transformado'] != '{}'
]

In [122]:
dfs_nr = []
for producto in productos_nr:
    prompt = generar_prompt_ollama(producto, etiquetas)
    respuesta = obtener_respuesta_ollama(prompt)
    print(producto)
    print("Respuesta del modelo:")
    print(respuesta["message"]["content"])
    print("\n------------------------\n")
    df_nr = procesar_respuesta(respuesta["message"]["content"], etiquetas)
    if df_nr is not None:
        dfs_nr.append(df_nr)
    else:
        print("No se pudo extraer la tabla.\n")

{'details': "{{'Pisada': 'Neutro • Supinador'}} {{'Peso del Corredor': 'Ideal hasta 85 Kg'}} {{'Amortiguación': 'Alta'}} {{'Ritmo De Carrera': 'Entre 4:30 y 6:00 min/Km'}} {{'Distancia Recomendada': 'Desde 5 hasta 42 Km'}} {{'Superficie': 'Asfalto'}}", 'description': [{'Comodidad y Amortiguación Superior': 'Las Brooks Ghost 15 se destacan por su amortiguación DNA Loft v2, que proporciona una sensación de suavidad y confort bajo los pies, perfecta para quienes buscan un calzado que se adapte a ritmos de carrera que van desde 4:30 min/km hasta 6:00 min/km. Con un drop de 11 mm, estas zapatillas están diseñadas para corredoras con pisada neutra o supinadora, ofreciendo un soporte equilibrado que facilita la transición en cada zancada.'}, {'Diseñadas para Ritmos y Distancias Variadas': 'Ideales para distancias que van desde los 5 km hasta el maratón, las Brooks Ghost 15 están construidas para soportar volúmenes de entrenamiento semanal que oscilan entre 20 km y más de 50 km. Su capacidad d

In [123]:
if dfs_nr:
    df_total_nr = pd.concat(dfs_nr, ignore_index=True)
else:
    print("No se pudo crear el DataFrame total.")
    
df_total_nr.head()

,cordones,textil exterior,textil interior,suela,peso y/o talla,eco diseñado si o no,color,identificador,nombre de deporte,genero Mujer/Hombre/MIXTO
0,Neutro • Supinador,Malla de ingeniería,Upper con materiales reciclados,Goma,Ideal hasta 85 Kg,Sí,Neutro,Brooks Ghost 15,Corredora,Mujer
1,Pisada,Neutro,null,MetaRocker,Ideal hasta 85 Kg,Si,Asfalto,Mach 6,Corre,Mujer
2,Neutro • Supinador,Null,Null,Null,Ideal hasta 80 Kg,No,Asfalto,Hoka Rincon 4,Corredor,Mujer
3,Neutro,Asfalto,Espuma ProFly+,Mediasuela,Ideal hasta 80 Kg,Sí,Neutro,Hoka Mach 5,Corredor,Hombre
4,Supinador,Neutro • Supinador,null,AhAR,Ideal hasta 85 Kg,null,Neutro • Supinador,Novablast 4 LE,Entre 5 y 42 Km,Mujer/Hombre/MIXTO


In [124]:
df_total_nr

,cordones,textil exterior,textil interior,suela,peso y/o talla,eco diseñado si o no,color,identificador,nombre de deporte,genero Mujer/Hombre/MIXTO
0,Neutro • Supinador,Malla de ingeniería,Upper con materiales reciclados,Goma,Ideal hasta 85 Kg,Sí,Neutro,Brooks Ghost 15,Corredora,Mujer
1,Pisada,Neutro,null,MetaRocker,Ideal hasta 85 Kg,Si,Asfalto,Mach 6,Corre,Mujer
2,Neutro • Supinador,Null,Null,Null,Ideal hasta 80 Kg,No,Asfalto,Hoka Rincon 4,Corredor,Mujer
3,Neutro,Asfalto,Espuma ProFly+,Mediasuela,Ideal hasta 80 Kg,Sí,Neutro,Hoka Mach 5,Corredor,Hombre
4,Supinador,Neutro • Supinador,null,AhAR,Ideal hasta 85 Kg,null,Neutro • Supinador,Novablast 4 LE,Entre 5 y 42 Km,Mujer/Hombre/MIXTO
5,Pisada pronadora,Pronador,Algunos puestos pueden tener materiales sintét...,Alta densidad,Ideal hasta 85 Kg,Sí,Negro/Rosa,GT-2000 13,Corredor,Mujer/Hombre/MIXTO
6,Neutro • Supinador,Neutro,Nylon o plástico,ASICS Grip + AHARPLUS,Ideal hasta 85 Kg,Si,Neutro,Glideride Max,Corredora,Hombre
7,Pisada pronadora,Asfalto,Mediasuela tech,Suela de goma HYBRID ASICSGRIP,Ideal desde 65 Kg en adelante,Si,Blanco,Kayano 31,Corredor,Hombre
8,Pronador,Algodón/Polipropileno,Malla técnica con poliéster y spandex,Suela de caucho,Ideal hasta 85 Kg,Si,Negro/Cream,GT-1000 13,Pronador,Hombre
